# Topic modelling - BERTopic on Interpellations (debates)


Made by: Elsa Kidman

[1] M. Grootendorst, BERTopic: Neural topic modeling with a class-based TF-IDF procedure, 2022. https://maartengr.github.io/BERTopic/index.html

In [ ]:
# To be able to run this file the following packages needs to be installed
# !pip3 install --upgrade tensorflow
# !pip3 install BERTopic

## Preprocessing
- From the questions remove to/from formalia. This only works since the dataset is between 2018-2022. It changes in the dataset of all the years.
- From each answer remove the name of the person asking and "Herr Talman" or "Fru Talman".
- Remove all empty "debatdag". Worth noticing is that the data contains empty answers. These are not removed but maybe should.
- Declare stopwords, which are then removed during the topic modelling with BERT.



In [15]:
import re
import string
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


# Download NLTK stopwords data
nltk.download('stopwords')
nltk.download('punkt')

# Removes from/to (av/till in swedish) text and names in the beginning of a question. Since each question begin the same, like for example: "av Eric Westroth (SD)\ntill Statsrådet Ida Karkiainen (S)"
def remove_names(question):
    # The text is split the by lines and then the first two lines are excluded
    lines = question.split('\n')[2:]

    # Join the remaining lines back into a single string
    new_text = '\n'.join(lines)
    return new_text

# Every answer in interpellations begins with the name of the person talking followed by either "Fru talman!" or "Herr talman!," which refers to the speaker in the riksdag.
# Since this doesn't provide any interessting information it's removed.
def remove_frutalman(text):
  # Remove "Fru talman!"
  index = text.find("Fru talman!")

  # Is "Fru talman!" present in the string?
  if index != -1:
      # Remove the words before and including "Fru talman!", since before is the name of the person asking
      result_string = text[index + len("Fru talman!"):]
      return result_string
  else:
      return text

# see remove_frutalman
def remove_herrtalman(text):
  # Remove "Herr talman!"
  index = text.find("Herr talman!")

  # Is "Herr talman!" present in the string?
  if index != -1:
      # Remove the words before and including "Herr talman!", since before is the name of the person asking
      result_string = text[index + len("Herr talman!"):]
      return result_string
  else:
      return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
import pickle
import json
import numpy as np
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

# Reduces outlier topics. These are often marked as the topic nr -1
def reduce_outliers(model, text, topics):
  # Reduce outliers
  new_topics = model.reduce_outliers(text, topics)
  # Update Topic Representation
  model.update_topics(text, topics=new_topics, vectorizer_model=vectorizer_model)
  return model

# Get list of Swedish stopwords
stopword_custom = stopwords.words('swedish')
stop_list = ["ska", "ske", "det", "vore"] # "se", "ge"
stopword_custom.extend(stop_list)
stop_words = set(stopword_custom)
stop_words = list(stop_words)

# define our CountVectorizer to remove stopwords OLD
vectorizer_model_old = CountVectorizer(stop_words=stop_words)

# Improved model 2
vectorizer_model = CountVectorizer(min_df=2, stop_words = stop_words, ngram_range=(1, 2))


In [42]:
f = open('../data/data_2018-09-09_2022-09-11/data_debates_2018-09-09_to_2022-09-11.txt')
data_init_debates = json.load(f)

In [18]:
# This displays that it exist some empty information in the data. Should it be removed or not?
empty_debattdag = []
empty_answers = []

for interpellation in data_init_debates:
  if interpellation['debattdag'] == '':
    empty_debattdag.append(interpellation['debattdag'])
  if len(interpellation['answer']) == 0:
    empty_answers.append(interpellation['answer'])

print(f"Number of empty debattdag (debate day): {len(empty_debattdag)}\n")
print(f"Number of empty answers: {len(empty_answers)}")

Number of empty debattdag (debate day): 391

Number of empty answers: 457


In [19]:
# Get the data ready for BERTopic
data_debates = []
questions_debates = []

answers_debates = []
answers_without_talman = []

data_debates_combined = []
data_debates_combined_striped = []


for debate in data_init_debates:
    #if debate['debattdag'] != '':
    question = debate['question']
    question2 = remove_names(question) # Remove formality
    questions_debates.append(question2)

    # Answers
    debate_answers = debate['answer']
    answers = []
    answers_without = []
    for answer in debate_answers:
      entry = answer['answer']
      answers.append(entry)

      answer2 = remove_frutalman(entry) # Remove formality (Fru talman!)
      answer3 = remove_herrtalman(answer2)
      answers_without.append(answer3)

    #combined_string = ''.join(answers)
    answers_debates.append(answers)

    answers_without_talman.append(answers_without)

    # a list of combined answers and questions without herr/fru talman
    combined_string = ''.join(answers_without)
    data_debates_combined_striped.append(question2+combined_string)

    # Combined answers
    combined_string2 = ''.join(answers)
    data_debates_combined.append(question+combined_string2)
    data_debates.append(debate)

In [20]:
len(answers_debates)
answers_debates[1]

['Anf.\xa0\xa061\xa0\xa0Statsrådet ANNA-CAREN SÄTHERBERG\xa0(S):Fru talman! Jens Holm har frågat klimat- och miljöministern om ministern avser att vidta omedelbara åtgärder för att minska skogens sårbarhet för klimat och biologisk mångfald, om ministern avser att vidta åtgärder för att långsiktigt säkra skydd för de kvarvarande gamla naturskogarna på grund av deras höga biologiska värden och stora kollager, om ministern avser att verka för skogsbruksmetoder baserade på den bästa vetenskapliga kunskapen samt om ministern avser att vidta ytterligare åtgärder för att stärka skogens alla värden, inklusive lokal och global klimatnytta, rening av luft och vatten, turism och rekreation.Arbetet inom regeringen är så fördelat att de frågor som tas upp i interpellationen huvudsakligen faller inom mitt ansvarsområde, och interpella\xadtionen besvaras därför av mig.Klimatförändringarna kan komma att innebära fler konsekvenser för skogsbruket. Det är därför viktigt att skogsskötseln planeras med rå

## BERTopic


#### BERTopic on answers

In [21]:
# Since datapoints in interpellations can have multiple answers
flattened_answers = [entry for answers in answers_without_talman for entry in answers]

model_a = BERTopic(language="swedish",
                 calculate_probabilities=True,
                 vectorizer_model=vectorizer_model)

topics_a, probs_a = model_a.fit_transform(flattened_answers)
model_a = reduce_outliers(model_a, flattened_answers, topics_a)

model_a.get_topic_info()

2023-12-15 19:05:13,721 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


,Topic,Count,Name,Representation,Representative_Docs
0,0,320,0_asylsökande_uppehållstillstånd_migrationsver...,"[asylsökande, uppehållstillstånd, migrationsve...",[ Det är intressant att höra Moderaterna tala ...
1,1,288,1_bostäder_bostadsmarknaden_bostad_unga,"[bostäder, bostadsmarknaden, bostad, unga, små...","[ Ola Johansson har frågat mig om jag, givet d..."
2,2,1051,2_statsrådet_finns_också_kommer,"[statsrådet, finns, också, kommer, vill, fråga...",[ På den sista frågan kan jag svara att det är...
3,3,363,3_utsläppen_utsläpp_jens_holm,"[utsläppen, utsläpp, jens, holm, minska, jens ...",[ Jag vill tacka miljö- och klimatminister Isa...
4,4,309,4_eu_kommissionen_sverige_ludvig,"[eu, kommissionen, sverige, ludvig, direktivet...",[Anf. 20 STEN BERGHEDEN (M):Herr ålderspresi...
...,...,...,...,...,...
245,245,15,245_polisen_inre utlänningskontroller_utlännin...,"[polisen, inre utlänningskontroller, utlänning...","[ Jag måste säga att jag blir förvånad. Nej, d..."
246,246,19,246_adam_adam marttinen_marttinen_grov våldtäkt,"[adam, adam marttinen, marttinen, grov våldtäk...",[ Till att börja med är det helt fel som Adam ...
247,247,10,247_polisen_kontroller_ekipage_trafikpolisen,"[polisen, kontroller, ekipage, trafikpolisen, ...","[ Tack, statsrådet Mikael Damberg, för svaret!..."
248,248,15,248_barn_146_förts utomlands_förts,"[barn, 146, förts utomlands, förts, sommarlov,...",[ Jag tycker att det är anmärkningsvärt att st...


In [28]:
# Save answers TODO: change location

model_a.save("/content/gdrive/MyDrive/IT5/to_submit/models/BERTopic_debates_answers", serialization="pickle")

with open("/content/gdrive/MyDrive/IT5/to_submit/models/BERTopic_debates_answers_topics", "wb") as fp:
  pickle.dump(topics_a, fp)

with open("/content/gdrive/MyDrive/IT5/to_submit/models/BERTopic_debates_answers_probs", "wb") as fp:
  pickle.dump(probs_a, fp)



2023-12-15 19:11:03,653 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


#### On questions

In [29]:
model_q = BERTopic(language="swedish",
                   calculate_probabilities=True,
                   vectorizer_model=vectorizer_model)

topics_q, probs_q = model_q.fit_transform(questions_debates)
model_q = reduce_outliers(model_q, questions_debates, topics_q)

model_q.get_topic_info()

2023-12-15 19:17:30,107 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


,Topic,Count,Name,Representation,Representative_Docs
0,0,239,0_poliser_fråga_polisen_vill,"[poliser, fråga, polisen, vill, regeringen, st...",[ \nDagens kriminalitet utgör ett hot både mot...
1,1,113,1_turkiet_utrikesminister_is_iran,"[turkiet, utrikesminister, is, iran, mänskliga...",[ \nDen 24 oktober i år kunde man läsa att en ...
2,2,99,2_sverige_uppehållstillstånd_personer_johansson,"[sverige, uppehållstillstånd, personer, johans...",[ \nEn del av de personer som fått avslag på s...
3,3,82,3_elever_skolan_lärare_anna ekström,"[elever, skolan, lärare, anna ekström, ekström...",[ \nRiksrevisionen riktar kritik mot regeringe...
4,4,66,4_arbetsförmedlingen_eva nordmark_eva_nordmark,"[arbetsförmedlingen, eva nordmark, eva, nordma...",[ \nRegeringen ska under 2020 införa utvecklin...
5,5,101,5_pandemin_covid_covid 19_19,"[pandemin, covid, covid 19, 19, regeringen, sv...",[ \nCovid-19-pandemin var i likhet med andra s...
6,6,58,6_arlanda_flygplats_flygplatser_swedavia,"[arlanda, flygplats, flygplatser, swedavia, fl...",[ \nVi har nu deltagit i två möten efter att n...
7,7,91,7_utsläpp_eu_sverige_miljö,"[utsläpp, eu, sverige, miljö, procent, svenska...",[ \nKonsultbolaget Sweco har på uppdrag av Sve...
8,8,57,8_el_elproduktion_kraftnät_svenska kraftnät,"[el, elproduktion, kraftnät, svenska kraftnät,...",[ \nPå två år har Skåne drabbats av inte en ut...
9,9,53,9_jakt_regeringen_jägare_vapen,"[jakt, regeringen, jägare, vapen, sportskyttar...",[ \nEU driver just nu på för utökat förbud mot...


In [30]:
# Save 

model_q.save("models/BERTopic_debates_questions", serialization="pickle")

with open("models/BERTopic_debates_questions_topics", "wb") as fp:
  pickle.dump(topics_q, fp)

with open("models/BERTopic_debates_questions_probs", "wb") as fp:
  pickle.dump(probs_q, fp)

2023-12-15 19:17:36,875 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


#### Combined

In [31]:
model_combined = BERTopic(language="swedish",
                          calculate_probabilities=True,
                          vectorizer_model=vectorizer_model)

topics_combined, probs_combined = model_combined.fit_transform(data_debates_combined_striped)

model_combined = reduce_outliers(model_combined, data_debates_combined_striped, topics_combined)

model_combined.get_topic_info()

2023-12-15 19:25:03,444 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


,Topic,Count,Name,Representation,Representative_Docs
0,0,163,0_polisen_poliser_samordningsnummer_också,"[polisen, poliser, samordningsnummer, också, k...",[ \nEn väl fungerande polis är en förutsättnin...
1,1,92,1_sverige_uppehållstillstånd_kommer_migrations...,"[sverige, uppehållstillstånd, kommer, migratio...",[ \nNär migrationskrisen 2015 tvingade regerin...
2,2,81,2_elever_skolan_lärare_skolor,"[elever, skolan, lärare, skolor, skola, barn, ...",[ \nSkolan ska vara en skyddad plats där barn ...
3,3,112,3_vård_hälso_finns_regeringen,"[vård, hälso, finns, regeringen, får, behov, a...",[ \nSedan Stefan Löfven blev statsminister har...
4,4,92,4_pandemin_regeringen_också_många,"[pandemin, regeringen, också, många, finns, an...",[ \nSverige har under hela pandemin valt sin e...
5,5,108,5_sverige_eu_också_kommer,"[sverige, eu, också, kommer, regeringen, vill,...",[ \nFörra året ökade Sveriges utsläpp med näst...
6,6,60,6_arbetsförmedlingen_jobb_kommer_också,"[arbetsförmedlingen, jobb, kommer, också, rege...","[ \nFör de flesta, i synnerhet bland dem som h..."
7,7,61,7_vargar_varg_djur_vargen,"[vargar, varg, djur, vargen, beslut, naturvård...",[ \nNaturvårdsverket rapporterade 2019 till EU...
8,8,51,8_flyget_bromma_arlanda_flygplats,"[flyget, bromma, arlanda, flygplats, flygplats...",[ \nBeslutet om den planerade utbyggnaden av A...
9,9,57,9_el_sverige_kraftnät_svenska kraftnät,"[el, sverige, kraftnät, svenska kraftnät, komm...",[ \nPå två år har Skåne drabbats av inte en ut...


In [32]:
# Save combined

model_q.save("/content/gdrive/MyDrive/IT5/to_submit/models/BERTopic_debates_combined", serialization="pickle")

with open("/content/gdrive/MyDrive/IT5/to_submit/models/BERTopic_debates_combined_topics", "wb") as fp:
  pickle.dump(topics_q, fp)

with open("/content/gdrive/MyDrive/IT5/to_submit/models/BERTopic_debates_combined_probs", "wb") as fp:
  pickle.dump(probs_q, fp)


2023-12-15 19:25:33,544 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


### Save to dataset file

In [34]:
# Get the topics for each entry.
document_topic_answers = model_a.get_document_info(flattened_answers)
document_topic_questions = model_q.get_document_info(questions_debates)

document_topic_combined = model_combined.get_document_info(data_debates_combined_striped)

In [ ]:
da = data_init_debates.copy()
for i, entry in enumerate(da):
  answers = entry['answer']
  if len(answers) != 0:
    for j, entry in enumerate(answers):
      ans = answer['answer']

In [43]:
columns_to_extract = ['Topic', 'Top_n_words']
new_data = []

for i, entry in enumerate(data_init_debates):
  answers = entry['answer']

  topic_id_combined, words_combined = document_topic_combined.loc[i, columns_to_extract].values
  words_combined = words_combined.split(' - ')
  new_element_combined = {'id_topic_combined': int(topic_id_combined), 'top_10_words_combined': words_combined}
  entry.update(new_element_combined)


  topic_id_q, words_q = document_topic_questions.loc[i, columns_to_extract].values
  words_q = words_q.split(' - ')

  new_element_q = {'id_topic_question': int(topic_id_q), 'top_10_words_question': words_q}

  entry.update(new_element_q)

  if len(answers) != 0:
    for j, answer in enumerate(answers):
      ans = answer['answer']
      topic_id_a, words_a = document_topic_answers.loc[(i+j), columns_to_extract].values
      words_a = words_a.split(' - ')

      answer['id_topic_answer'] = int(topic_id_a)
      answer['top_10_words'] = words_a


In [44]:
data_init_debates[3]

{'id_': 'h910259',
 'question': 'av Mikael Larsson (C)\ntill Infrastrukturminister Tomas Eneroth (S)\n\xa0\nUnder våren 2022 ska den nationella transportplanen för 2022–2033 beslutas om av regeringen. Investeringar i nya järnvägar och vägar, samt underhåll av det vi redan har, ska göras kommande 12-årsperiod med nästan 900 miljarder kronor. \nTransportinfrastruktur är något som påverkar människor alla dagar i veckan och året om. Alla vill ha bra vägar\xa0och\xa0järnvägar, kunna använda flyget och sjöfarten och ha en fungerade mobiltäckning, oavsett om det är på landsbygden eller i städerna. \nI Västra Götaland sker just nu ombyggnad av E20 i Skaraborg, ny sträckning av riksväg\xa041 mellan Björketorp och Sundholmen, Viskadalsbanan renoveras för att klara av framtidens tåg och flera mindre statliga vägar på landsbygden får ny asfalt. Allt detta är välkommet och en viktig del för att Sverige ska kunna utvecklas. När ny infrastruktur som dessa projekt ska byggas eller byggas om krävs en b

In [45]:
type(data_init_debates[0]['id_topic_question'])

int

In [46]:
import json

with open("data/data_2018-09-09_2022-09-11/data_debates_topics_updated.json", "w") as fp:
    json.dump(data_init_debates, fp)